In [1]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [10]:
%%cython -a
# distutils: language = c++

from libcpp.vector cimport vector
from cythonic.core.ant cimport ant_state
import sqlite3
import json
# from cythonic.core.visualization import StigmergyPlot
import numpy as np
cimport numpy as cnp
from cythonic.core.sim_controller cimport Sim
from cythonic.plugins import queries
from cythonic.plugins.dummy_dicts import ant_dict, queen_dict, domain_dict, gauss_dict, sim_dict, deposit_dict, sens_dict

cdef class db_controller():
    cdef object db
    cdef str pending_qry
    
    def __cinit__(self, str db_path,str db_name):
        " class constructor "
        if not db_path[-1] =='/':
            db_path += '/'
        self.db = sqlite3.connect(db_path+db_name)
        
    cdef unsigned int new_sim_id(self,):
        " start logging of sim in sqlite database, get ID of simulation "
        cdef object cursor = self.db.cursor()
        cdef str qry = queries.new_sim()
        cdef unsigned int id
        try:
            cursor.execute(qry)
            id = cursor.lastrowid
            self.db.commit()
        except Exception as error:
            self.db.close()
            raise error
        return id
    
    cdef void executemany(self,str qry, list values):
        " use the executemany for bulk insert "
        cursor = self.db.cursor()
        try:
            cursor.executemany(qry,values)
            self.db.commit()
        except Exception as error:
            self.db.close()
            raise error
    
    cdef void execute(self, str qry ):
        " execute an INSERT INTO query "
        cdef object cursor = self.db.cursor()
        try:
            cursor.execute(qry)
            self.db.commit()
        except Exception as error:
            self.db.close()
            raise error
        
    def close(self):
        self.db.close()
        
    def print_all(self,str qry):
        " Print all results from a select qry "
        cursor = self.db.cursor()
        cursor.execute(qry)
        rows = cursor.fetchall()
        for row in rows:
            print(row)
            
#     cdef void add_rows(self, bytes qry):
#         " add rows to a query "
#         self.pending_qry+= qry


cdef class sim_recorder(Sim):
    cdef db_controller db
    cdef:
        unsigned int update_interval
        unsigned int id
        str pending_qry
        list qry_args
        dict ant_dict, queen_dict, domain_dict, gauss_dict, sim_dict, deposit_dict, sens_dict
    def __init__(self,queen_args, domain_args, sim_args):
        """ Need to store the dicts for inserting into database """
        self.queen_dict = queen_args
        self.sim_dict = sim_args
        self.domain_dict = domain_args
        self.ant_dict=queen_args['ant_dict']
        super(sim_recorder,self).__init__(queen_args = queen_args, domain_args = domain_args, **sim_args)
    
    def init_connection(self,str path, str dbname):
        self.db = db_controller(path,dbname)
        
    def setup_sim(self, str deposit_style, dict deposit_dict, dict gauss_dict, unsigned int upload_interval):
        " prepare the simulation "
        self.update_interval = upload_interval
        self.set_depositing(deposit_style, deposit_dict )
        self.set_gaussian(**gauss_dict)
        self.id = self.db.new_sim_id()
        self.push_settings(deposit_dict, gauss_dict)
        self.flush_resultset() # initialize the insert state query
        
    def push_settings(self, dict deposit_dict, dict gauss_dict):
        """ INSERT the simulation settings into the database prior to experiment """
        # manipulation, convert nested dict to string for database
        cdef dict sim_dict = self.sim_dict.copy()
        sim_dict['deploy_timing_args'] = json.dumps(self.sim_dict['deploy_timing_args']).replace("'","''")
    
        queries_list =(
            queries.insert_ant(self.id, **self.ant_dict),
            queries.insert_queen(self.id,**self.queen_dict),
            queries.insert_domain(self.id,**self.domain_dict),
            queries.insert_gauss(self.id,**gauss_dict),
            queries.insert_deposit(self.id,**deposit_dict),
            queries.insert_sens(self.id,**ant_dict['sens_dict']),
            queries.sim_settings(self.id,**sim_dict), # use the local manipulated copy of sim_dict
                    )
        for qry in queries_list:
            self.db.execute(qry)
        
    cdef void flush_resultset(self,):
        " prepare for a new batch of settings "
#         self.pending_qry = queries.insert_stepupdates()
        self.qry_args = []
        self.pending_qry = "INSERT INTO STEP (SIM_ID, STEP_NR, ANT_ID, X, Y, THETA, Q) VALUES (?,?,?,?,?,?,?)"
        
    cdef void extract_antstate(self, unsigned int step):
        " make a copy of the vital ant state elements and store "
        cdef unsigned int i
        for i in range(self.queen.count_active):
            " push the state to a list "
            self.qry_args.append((self.id, step, self.queen.state_list[i].id, self.queen.state_list[i].pos.x,
                                  self.queen.state_list[i].pos.y, self.queen.state_list[i].theta,
                                  self.queen.drop_quantity[i]))
       
    cdef void run_sim(self, bint record):
        " run the simulation "
        cdef unsigned int i
        self.db.execute(queries.update_sim(self.id, status = 'STARTED'))
        for i in range(self.steps):
            self.sim_step()
            if record:
                self.extract_antstate(i)
            if i>0 and record and (i == self.steps-1 or i%self.update_interval == 0):
#                 self.db.execute(self.pending_qry[:-1]) # omit the last comma in the query
                self.db.executemany(self.pending_qry,self.qry_args)
                self.flush_resultset() # start with a fresh query
        self.db.execute(queries.update_sim(self.id, status = 'FINISHED'))
        print(f" \n {self.foodcount} ants returned to the nest ")
        print(f" \n {self.nestcount} ants found the food in the first place \n")
            
    
from time import time
sim_dict['steps'] = 3000
sim_dict['n_agents'] = 100
domain_dict['pitch'] = 2
cdef double toc, tic
cdef sim_recorder simulator = sim_recorder(queen_args=queen_dict, domain_args = domain_dict, sim_args=sim_dict)
tic = time()
simulator.init_connection('/home/bram/ANTS/cythonic/database/','stigmergy.db')
simulator.setup_sim('constant', deposit_dict, gauss_dict, 500,)
simulator.run_sim(True)
simulator.db.close()

toc = time()
print(f"It took a whopping {(toc-tic)*1000} msec")

 
 0 ants returned to the nest 
 
 0 ants found the food in the first place 

It took a whopping 4650.604724884033 msec
